Idea of this notebook:
- separate keys with process_audio_files function
- ideally for both datasets (practical_dl and MKA)
- remember optimal tresholds and do some majority voting in future data loader

save separated audio in folder structure:
- processed data
    - train/val/test
        - label 
            - segmented recordings
- later add there also files from https://www.kaggle.com/datasets/nguyncaoduy/keystroke-noiseless-final

Idea for DataLoader:
- Separate audio for 10ms frames
- Calculate energylevel in each frame, if the treshold is exceeded then enlargen the frame with subsequent 90ms or so

In [9]:
import os
from tqdm import tqdm
import shutil
from sklearn.model_selection import train_test_split

from src.utils.data_engineering_utils import process_audio_files

### `MKA` and `Practical_dl` datasets preprocessing

In [ ]:
OUTPUT_DIR = "./data/final/"
os.makedirs(OUTPUT_DIR, exist_ok=True)
OUTPUT_DIR_IMG = "./data/final_img/"
os.makedirs(OUTPUT_DIR_IMG, exist_ok=True)

DATA_DIR = './data/raw_data/MKA/Mac/Raw Data/'
DATASET = 'mka'

process_audio_files(DATA_DIR, OUTPUT_DIR, OUTPUT_DIR_IMG, DATASET, True)

DATA_DIR = './data/raw_data/practical_dl/'
DATASET = 'practical'
process_audio_files(DATA_DIR, OUTPUT_DIR, OUTPUT_DIR_IMG, DATASET, True)


Processing files in .git: 100%|██████████| 5/5 [00:00<00:00, 275941.05it/s]
Processing files in objects: 0it [00:00, ?it/s]
Processing files in pack: 100%|██████████| 3/3 [00:00<00:00, 167772.16it/s]
Processing files in info: 0it [00:00, ?it/s]
Processing files in logs: 100%|██████████| 1/1 [00:00<00:00, 67650.06it/s]
Processing files in refs: 0it [00:00, ?it/s]
Processing files in heads: 100%|██████████| 1/1 [00:00<00:00, 55188.21it/s]
Processing files in remotes: 0it [00:00, ?it/s]
Processing files in hooks: 100%|██████████| 14/14 [00:00<00:00, 932067.56it/s]
Processing files in refs: 0it [00:00, ?it/s]
Processing files in heads: 100%|██████████| 1/1 [00:00<00:00, 50533.78it/s]
Processing files in tags: 0it [00:00, ?it/s]
Processing files in remotes: 0it [00:00, ?it/s]
Processing files in origin: 100%|██████████| 1/1 [00:00<00:00, 12336.19it/s]


### `Noiseless` Dataset

In [6]:
SOURCE_DIR = './data/raw_data/data_noiseless_final/'
DATA_DIR_NOISELESS = './data/raw_data/noiseless/'

labels_in_dir_new = set()
for dirname, _, filenames in os.walk(SOURCE_DIR):
    for filename in filenames:
        if filename.endswith('.wav'):
            label = filename.split('-')[1]
            label_dir = os.path.join(DATA_DIR_NOISELESS, label)
            if not os.path.exists(label_dir):
                os.makedirs(label_dir)

            src_path = os.path.join(dirname, filename)
            filename_new = 'mac_live_' + label + '_' + filename.split('-')[-1].split('.')[0] + '.wav'
            dst_path = os.path.join(label_dir, filename_new)

            shutil.copy(src_path, dst_path)

In [11]:
def move_files(file_paths, subset_name):
    for file_path in file_paths:
        label = file_path.split('/')[-2]

        if not os.path.exists(os.path.join(OUTPUT_DIR, 'noiseless', subset_name, label)):
            os.makedirs(os.path.join(OUTPUT_DIR, 'noiseless', subset_name, label), exist_ok=True)

        file_name = os.path.basename(file_path)
        dest_path = os.path.join(OUTPUT_DIR, 'noiseless', subset_name, label, file_name)

        shutil.move(file_path, dest_path)

In [12]:
for key in os.listdir(DATA_DIR_NOISELESS):
    key_dir = os.path.join(DATA_DIR_NOISELESS, key)
    if os.path.isdir(key_dir):
        files = [os.path.join(key_dir, f) for f in os.listdir(key_dir) if f.endswith('.wav')]
        train_files, temp_files = train_test_split(files, test_size=0.3, random_state=42)
        val_files, test_files = train_test_split(temp_files, test_size=0.5, random_state=42)

        move_files(train_files, 'train')
        move_files(val_files, 'val')
        move_files(test_files, 'test')